In [7]:

# Load Model
import joblib

model_path = "/content/drive/MyDrive/AI Saturdays /final_random_forest_model.pkl"

model = joblib.load(model_path)

print("✅ Model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


✅ Model loaded successfully!


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
type(model)

sklearn.ensemble._forest.RandomForestClassifier

In [9]:
# Load dataset
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/AI Saturdays /lagos_prepared_with_features.csv')
df.head()

,uuid,City,Surface_area,Potential_installable_area,Peak_installable_capacity,Energy_potential_per_year,Assumed_building_type,Estimated_tilt,Estimated_building_height,Estimated_capacity_factor,Area_utilization_ratio,Energy_density,Capacity_density,System_efficiency,Energy_category
0,2c2e6093-4675-4342-8141-78b7da7319f0,Lagos,174.40,103.42,20.06,26229.95,single family residential,13.33,5.05,1307.39,0.593005,253.625508,0.193966,0.149267,Medium
1,94e9aea9-4208-4a77-8578-100648bbca1a,Lagos,627.00,463.14,89.85,118061.12,multi-family residential,6.34,5.74,1314.00,0.738660,254.914540,0.194002,0.149998,High
2,91841e20-5844-4aa5-9ba8-7c27b174ab4e,Lagos,418.81,309.52,60.05,78901.84,multi-family residential,8.41,8.75,1314.00,0.739046,254.916774,0.194010,0.149993,High
3,2818b582-c998-4c09-b473-e79de8caa898,Lagos,548.53,409.27,79.40,104328.57,multi-family residential,6.32,5.60,1314.00,0.746121,254.913798,0.194004,0.149996,High
4,32cfd9be-33f3-4761-a48a-a22077dcecc0,Lagos,485.97,359.03,69.65,89546.93,multi-family residential,17.48,5.27,1285.63,0.738790,249.413503,0.193995,0.146766,High


In [11]:
print(df['Assumed_building_type'].unique())

['single family residential' 'multi-family residential'
 'peri-urban settlement' 'commercial' 'industrial' 'public']


In [12]:

# Inspect Dataset

df.head().columns

Index(['uuid', 'City', 'Surface_area', 'Potential_installable_area',
       'Peak_installable_capacity', 'Energy_potential_per_year',
       'Assumed_building_type', 'Estimated_tilt', 'Estimated_building_height',
       'Estimated_capacity_factor', 'Area_utilization_ratio', 'Energy_density',
       'Capacity_density', 'System_efficiency', 'Energy_category'],
      dtype='object')

In [14]:
# Define the expected categorical options
building_types = [
'single family residential', 'multi-family residential',
'peri-urban settlement', 'commercial', 'industrial', 'public'
]

# Prediction function
def predict_energy_category(estimated_tilt, estimated_building_height,
                             estimated_capacity_factor, area_utilization_ratio,
                             capacity_density, assumed_building_type):

    # Create DataFrame with one-hot encoded building type
    input_dict = {
        "Estimated_tilt": [estimated_tilt],
        "Estimated_building_height": [estimated_building_height],
        "Estimated_capacity_factor": [estimated_capacity_factor],
        "Area_utilization_ratio": [area_utilization_ratio],
        "Capacity_density": [capacity_density],
    }

    # Add one-hot encoding columns manually
    for btype in building_types[1:]:  # drop_first=True used in training
        col_name = f"Assumed_building_type_{btype}"
        input_dict[col_name] = [1 if assumed_building_type == btype else 0]

    input_df = pd.DataFrame(input_dict)

    # Make prediction
    pred = model.predict(input_df)[0]
    return f"Predicted Energy Category: {pred}"

# Gradio Interface
import gradio as gr

iface = gr.Interface(
    fn=predict_energy_category,
    inputs=[
        gr.Number(label="Estimated Tilt"),
        gr.Number(label="Estimated Building Height"),
        gr.Number(label="Estimated Capacity Factor"),
        gr.Number(label="Area Utilization Ratio"),
        gr.Number(label="Capacity Density"),
        gr.Dropdown(building_types, label="Assumed Building Type"),
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="🏙️ Lagos Building Solar Energy Efficiency Predictor",
    description="Predicts whether a building in Lagos has Low, Medium, or High solar energy efficiency potential."
)

# Launch inside Colab
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7edb5bbcf909bd03f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
